In [256]:
# --------------------------------------------
# IMPORT LIBRARIES
# --------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', None)

In [257]:
# --------------------------------------------
# PARAMETRES DE SIMULATION
# --------------------------------------------
Nobs = 10000 # nombre d'observations
NX = 6 # nombre de covariables
PX = 1/2 # probabilité qu'une covariable vaille 1
alpha_tau = 0.5 #effet du traitement

alpha_eZ = [1,1,-1,0,0,0] #effet des covariables sur le propensity score
alpha0_eZ = -np.sum(alpha_eZ)*PX #intercept du propensity score
alpha_eY = [-1,-1,1,0,0,0] #effet des covariables sur la variable d'intérêt
alpha0_eY = -np.sum(alpha_eY)*PX #intercept de la variable d'intérêt

# --------------------------------------------
# GENERATION DES COVARIABLES X
# --------------------------------------------
X = np.random.binomial(1,PX,(Nobs,NX))

# --------------------------------------------
# GENERATION DE Z
# --------------------------------------------
def sigmoid(x): # Définition de la fonction sigmoïde
    return 1 / (1 + np.exp(-x))

eZ = sigmoid(X.dot(alpha_eZ) + alpha0_eZ) # Génération du propensity score
Z = np.random.binomial(1,eZ)

# --------------------------------------------
# GENERATION DE Y
# --------------------------------------------
eY = sigmoid(X.dot(alpha_eY) + alpha_tau*Z + alpha0_eY)
Y = np.random.binomial(1,eY)

# --------------------------------------------
# Calcul de P(Y(1)=1) et P(Y(0)=1)
# --------------------------------------------
Z1 = np.ones(Nobs)
Z0 = np.zeros(Nobs)

EY1 = sigmoid(X.dot(alpha_eY) + alpha_tau*Z1 + alpha0_eY)
EY0 = sigmoid(X.dot(alpha_eY) + alpha_tau*Z0 + alpha0_eY)

# --------------------------------------------
# MISE EN FORME ET EXPORT DES DONNEES
# --------------------------------------------
df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(NX)])  # noms des colonnes X1, X2, ...
df['Y'] = Y
df['Z'] = Z
df['EY1'] = EY1
df['EY0'] = EY0

# Réorganiser les colonnes pour avoir Y, Z, puis les X
df = df[['Y', 'EY1', 'EY0', 'Z'] + [f"X{i+1}" for i in range(NX)]]

# Exporter le DataFrame au format CSV
#df.to_csv("data.csv", index=False)

In [258]:
# --------------------------------------------
# ANALYSE DES DONNEES
# --------------------------------------------
# Calculer la moyenne de Y pour chaque valeur de Z
mean_Y_by_Z = df.groupby('Z')['Y'].mean()

# Afficher les résultats
print('E[Y,Z=1] - E[Y,Z=0] =', round(mean_Y_by_Z[1]-mean_Y_by_Z[0],ndigits=3))   
print('E[Y(1)] - E[Y(0)] =', round(df['EY1'].mean()-df['EY0'].mean(),ndigits=3))

# Calculer la moyenne de Y pour chaque combinaison de Z et X1
#mean_Y_by_Z_X1_X2_X3 = df.groupby(['Z', 'X1', 'X2', 'X3'])['Y'].mean()

# Afficher les résultats
#print(mean_Y_by_Z_X1_X2_X3)

E[Y,Z=1] - E[Y,Z=0] = -0.049
E[Y(1)] - E[Y(0)] = 0.105
